In [10]:
#ライブラリのインポート
import glob
import pandas as pd
import pandas_profiling as pdp
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae
import df_pre as df_pre

#lgb用
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

from sklearn.model_selection import KFold

In [11]:
#ファイルの読み込み
files = glob.glob("../../data/train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
data_list
df = pd.concat(data_list)

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
#変換処理の適用
df = df_pre.data_pre(df)

test


In [13]:
#lgbの実行
df_train, df_val =train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

lgb_train = lgb.Dataset(train_x, train_y)
lgb_eval = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

categorical_features = ["地区名", "最寄駅：名称", "間取り", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]
model = lgb.train(params, lgb_train, valid_sets=lgb_eval,
                  num_boost_round=1000,
                  early_stopping_rounds=100,
                  categorical_feature = categorical_features)


C:\Users\user\anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['今後の利用目的', '取引の事情等', '地区名', '改装', '最寄駅：名称', '用途', '都市計画', '間取り']
  _log_warning('categorical_feature in Dataset is overridden.\n'


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.031116 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11641
[LightGBM] [Info] Number of data points in the train set: 554305, number of used features: 17
[LightGBM] [Info] Start training from score 7.224308
[1]	valid_0's l1: 0.253878


C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\user\anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.237522
[3]	valid_0's l1: 0.223631
[4]	valid_0's l1: 0.210979
[5]	valid_0's l1: 0.200432
[6]	valid_0's l1: 0.191034
[7]	valid_0's l1: 0.182857
[8]	valid_0's l1: 0.174873
[9]	valid_0's l1: 0.168568
[10]	valid_0's l1: 0.162554
[11]	valid_0's l1: 0.1571
[12]	valid_0's l1: 0.152635
[13]	valid_0's l1: 0.148056
[14]	valid_0's l1: 0.14402
[15]	valid_0's l1: 0.140454
[16]	valid_0's l1: 0.137259
[17]	valid_0's l1: 0.134485
[18]	valid_0's l1: 0.132166
[19]	valid_0's l1: 0.129989
[20]	valid_0's l1: 0.127361
[21]	valid_0's l1: 0.125316
[22]	valid_0's l1: 0.122847
[23]	valid_0's l1: 0.121022
[24]	valid_0's l1: 0.119189
[25]	valid_0's l1: 0.117425
[26]	valid_0's l1: 0.115449
[27]	valid_0's l1: 0.114203
[28]	valid_0's l1: 0.112866
[29]	valid_0's l1: 0.111434
[30]	valid_0's l1: 0.110363
[31]	valid_0's l1: 0.109328
[32]	valid_0's l1: 0.108303
[33]	valid_0's l1: 0.107297
[34]	valid_0's l1: 0.106441
[35]	valid_0's l1: 0.1057

In [14]:
va_pred = model.predict(val_x)
score = mae(val_y, va_pred)
print(f'mae:{score:.6f}')

mae:0.076810


In [15]:
#提出用データ作業
df_test = pd.read_csv("../../data/test.csv", index_col=0)
df_test = df_pre.data_pre(df_test)
df_test = df_test.drop(columns="取引価格（総額）_log", axis=1)

predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("../../data/submit_test.csv")



test
